# Alunos

| Nome                        | Matrícula |
|-----------------------------|-----------|
| Luiz Filipe Bartelega Penha | 202111082 |
| Vitor Pires Zini            | 202110169 |

# Problema 3

 A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprova uma melhoria da base da cobrança de impostos que prevê a condenação de uma área habitacional do centro da cidade e sua substituição por um conjunto habitacional moderno.
  > O projeto envolve duas fases:
  1. demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto;
  2. construção do novo conjunto urbano. A seguir daremos um resumo da situação.

Um total de 300 casas aquém do padrão podem ser demolidas. Cada casa ocupa um lote de 0,25 acres. O custo de demolição de uma casa condenada é de \$2.000. Os tamanhos de lotes para domicílios (unidades) simples, duplos, triplos e quádruplos são de 0,18; 0,28; 0,4 e 0,5 acres, respectivamente. Ruas, espaços abertos e instalações públicas ocupam 15% da área disponível.
 No novo conjunto habitacional, as unidades triplas e quádruplas representam no mínim 25% do total. Unidades simples devem representar no mínimo 20% de todas as unidades, e unidades duplas, no mínimo 10%. O imposto cobrado por unidade para unidades simples, duplas, triplas e quádruplas é de \$1.000, \$1.900, \$2.700 e \$3.400, respectivamente. O custo de construção por unidade domiciliar simples, dupla, tripla e quádrupla é de \$50.000, \$70.000, \$130.000 e \$160.000, respectivamente. O financiamento acordado com um banco local será de no máximo $15 milhões. Quantas unidades de cada tipo devem ser construídas para maximizar a arrecadação de impostos?


# Modelagem matemática

A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão.

## Declaração dos parâmetros

* $U$: conjunto de unidades domiciliares.
* $c_{u} \in \mathbb{R}_+$: custo de construção de uma unidade $u \in U$.  
* $i_{u} \in \mathbb{R}_+$: imposto de uma unidade $u \in U$.  
* $a_{u} \in \mathbb{R}_+$: area de uma unidade $u \in U$ em acres.
* **Área destinada a unidades domiciliares: $100\% - 15\% == 85\%$**
* **Área disponível para construção: $63.75$ acres $== 85\%$ da área disponível**
* **Financiamento da obra: $\$15.000.000$**.
* **Custo de demolição das casas antigas: $300 ⋅ \$2000 = \$600.000$**


In [1]:
tipos_unidades = [1, 2, 3, 4]

custos_de_construcao = {
  1: 50000,
  2: 70000,
  3: 130000,
  4: 160000,
}

impostos = {
  1: 1000,
  2: 1900,
  3: 2700,
  4: 3400,
}

areas = {
  1: 0.18,
  2: 0.28,
  3: 0.4,
  4: 0.5,
}


## Variáveis de decisão

* $x_{u} \ge 0$: quantidade de unidades do tipo $u \in U$.

## Função objetivo

* Maximização da arrecadação dos impostos de unidades domiciliares: $\max \sum_{u \in U} (i_{u}x_{u})$

## Restrições

- Restrição 1: A construção deve atender o financiamento

\begin{equation}
\sum_{u \in U} x_u*c_u \le 14.400.000  
\end{equation}

- Restrição 2: Restrição de área
\begin{equation}
\sum_{u \in U} x_u*a_u \le 85\% * Área
\end{equation}

- Restrição 3: Cada tipo de domicílio deve representar uma porcentagem do total de domicílios

\begin{equation}
x_3 + x_4 \ge 25\% * Total
\end{equation}
\begin{equation}
x_1 \ge 20\% * Total
\end{equation}
\begin{equation}
x_2 \ge 10\% * Total
\end{equation}


#Resolvendo o Problema

## Instalação da biblioteca PuLP

In [2]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 39.8 MB/s eta 0:00:00


## Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem('arrecadacao_impostos', pulp.LpMaximize)

## Variáveis de decisão

In [4]:
quantidade_unidades = pulp.LpVariable.dicts('unidades', tipos_unidades, lowBound=0, cat='Integer')

## Função objetivo

In [5]:
modelo += pulp.lpSum([quantidade_unidades[tipo_unidade] * impostos[tipo_unidade]] for tipo_unidade in tipos_unidades)

## Restrições

- Restrição 1: A construção deve atender o financiamento

In [6]:
soma = 0
for tipo_unidade in tipos_unidades:
  soma += quantidade_unidades[tipo_unidade]*custos_de_construcao[tipo_unidade]

modelo += soma <= 14400000 # 15 milhões - Custo de demolição (600.000)

- Restrições 2 e 3: Restrições de área e mínimo de domicílios

In [7]:
soma_area = 0
quantidade_total_domilicios = 0

for tipo_unidade in tipos_unidades:
  soma_area += quantidade_unidades[tipo_unidade]*areas[tipo_unidade]
  quantidade_total_domilicios += quantidade_unidades[tipo_unidade]

modelo += soma_area <= 300 * 0.25 * 0.85 # 85% da área disponível após a demolição das casas.

modelo += quantidade_unidades[3]+quantidade_unidades[4] >= 0.25 * quantidade_total_domilicios # 25% do total

modelo += quantidade_unidades[1] >= 0.2 * quantidade_total_domilicios # 20% do total

modelo += quantidade_unidades[2] >= 0.1 * quantidade_total_domilicios # 10% do total

### Resolvendo o problema

In [8]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [9]:
print('status: ', pulp.LpStatus[status])
print('funcao objetivo: ', modelo.objective.value())
for tipo in tipos_unidades:
  print('\nUnidade:', tipo)
  print('Quantidade de unidades:', quantidade_unidades[tipo].value())
  print('Área ocupada:', round(quantidade_unidades[tipo].value()*areas[tipo], 2), 'acres')
  print('Custo de construção desse tipo:', round(quantidade_unidades[tipo].value()*custos_de_construcao[tipo], 2))
  print('Imposto arrecadado por esse tipo:', round(quantidade_unidades[tipo].value()*impostos[tipo], 2))

status:  Optimal
funcao objetivo:  341100.0

Unidade: 1
Quantidade de unidades: 35.0
Área ocupada: 6.3 acres
Custo de construção desse tipo: 1750000.0
Imposto arrecadado por esse tipo: 35000.0

Unidade: 2
Quantidade de unidades: 96.0
Área ocupada: 26.88 acres
Custo de construção desse tipo: 6720000.0
Imposto arrecadado por esse tipo: 182400.0

Unidade: 3
Quantidade de unidades: 37.0
Área ocupada: 14.8 acres
Custo de construção desse tipo: 4810000.0
Imposto arrecadado por esse tipo: 99900.0

Unidade: 4
Quantidade de unidades: 7.0
Área ocupada: 3.5 acres
Custo de construção desse tipo: 1120000.0
Imposto arrecadado por esse tipo: 23800.0
